In [1]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import plotly.express as px
import pandas as pd
import re
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [3]:
import unicodedata

In [4]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [5]:
import selenium
from selenium import webdriver

In [6]:
from selenium.webdriver.support.ui import WebDriverWait

In [7]:
import time

In [8]:
cEXT = pickle.load( open( "C:/Users/i.adamlyuk/Downloads/cEXT.p", "rb"))
cNEU = pickle.load( open( "C:/Users/i.adamlyuk/Downloads/cNEU.p", "rb"))
cAGR = pickle.load( open( "C:/Users/i.adamlyuk/Downloads/cAGR.p", "rb"))
cCON = pickle.load( open( "C:/Users/i.adamlyuk/Downloads/cCON.p", "rb"))
cOPN = pickle.load( open( "C:/Users/i.adamlyuk/Downloads/cOPN.p", "rb"))
vectorizer_31 = pickle.load( open( "C:/Users/i.adamlyuk/Downloads/vectorizer_31.p", "rb"))
vectorizer_30 = pickle.load( open( "C:/Users/i.adamlyuk/Downloads/vectorizer_30.p", "rb"))

C:\ProgramData\Anaconda\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.1 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\ProgramData\Anaconda\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.1 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\ProgramData\Anaconda\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.1 when using version 1.2.1. This might lead to breaking code or invalid results.

In [9]:
def predict_personality(text):
    scentences = re.split("(?<=[.!?]) +", text)
    text_vector_31 = vectorizer_31.transform(scentences)
    text_vector_30 = vectorizer_30.transform(scentences)
    EXT = cEXT.predict(text_vector_31)
    NEU = cNEU.predict(text_vector_30)
    AGR = cAGR.predict(text_vector_31)
    CON = cCON.predict(text_vector_31)
    OPN = cOPN.predict(text_vector_31)
    return [EXT[0], NEU[0], AGR[0], CON[0], OPN[0]]


In [10]:
def predict_personality_ext(text):
    scentences = re.split("(?<=[.!?]) +", text)
    text_vector_31 = vectorizer_31.transform(scentences)
    EXT = cEXT.predict(text_vector_31)
    return EXT[0]

In [11]:
def predict_personality_neu(text):
    scentences = re.split("(?<=[.!?]) +", text)
    text_vector_30 = vectorizer_30.transform(scentences)
    NEU = cNEU.predict(text_vector_30)
    return NEU[0]

In [12]:
def predict_personality_agr(text):
    scentences = re.split("(?<=[.!?]) +", text)
    text_vector_31 = vectorizer_31.transform(scentences)
    AGR = cAGR.predict(text_vector_31)
    return AGR[0]

In [13]:
def predict_personality_con(text):
    scentences = re.split("(?<=[.!?]) +", text)
    text_vector_31 = vectorizer_31.transform(scentences)
    CON = cCON.predict(text_vector_31)
    return CON[0]

In [14]:
def predict_personality_opn(text):
    scentences = re.split("(?<=[.!?]) +", text)
    text_vector_31 = vectorizer_31.transform(scentences)
    OPN = cOPN.predict(text_vector_31)
    return OPN[0]

In [15]:
def remove_control_characters(s):
    s = unicodedata.normalize("NFKD", s)
    return "".join(ch for ch in s if unicodedata.category(ch)[0]!="C")

# очищаем код от выбранных элементов
def delete_div(code, tag, arg):
    # находим все указанные теги с параметрами
    for div in code.find_all(tag, arg):
        # и удаляем их из кода
        div.decompose()


def clear_text(url):
    # получаем исходный код страницы
    inner_html_code = str(urlopen(url).read(), 'utf-8')

    # отправляем исходный код страницы на обработку в библиотеку
    inner_soup = BeautifulSoup(inner_html_code, "html.parser")
    
    # оставляем только блок с содержимым статьи
    inner_soup = inner_soup.find('div', {"id": 'content'})

    # удаляем ненужное
    delete_div(inner_soup, "p", {'class': 'content-summary'})
    delete_div(inner_soup, "div", {'class': 'hidden-computer-only'})
    delete_div(inner_soup, "div", {'class': 'newsletter-subscribe-card'})
    
    # ищем статус проекта успешный/неудачный 
    status = BeautifulSoup(inner_html_code, "html.parser").find('div', {'class':"w-condition-invisible"}).attrs.get('class')[0]
    if status == 'section-interview-failure':
        status = 1 #success
    else:
        status = 0 #fail  
    return (remove_control_characters(inner_soup.get_text()), status)


def find_articles(inner_html_code, max_size):
    # получаем исходный код страницы
    position = 0
    urls = []
#     inner_html_code = str(urlopen(url).read(), 'utf-8')
#     inner_soup = BeautifulSoup(inner_html_code, "html.parser")
    block = inner_html_code.find('div', {'class': 'w-dyn-items'})
    for a_tag in block.findAll("a"):
        if position == max_size:
            break
        position += 1
        href = a_tag.attrs.get("href").split('/')[-1]
        urls.append(href)
    print("len = ", len(urls))
    return urls

In [16]:
main_url = 'https://www.failory.com'
interviews_href = main_url + '/interviews'
one_interview_href = main_url + '/interview/'
size = 300
# driver = webdriver.PhantomJS(executable_path='phantomjs.exe')
# path = "C:\Users\89165\DidenkoResearch\personality-prediction-from-text\chromedriver_win32\chromedriver.exe"
# driver = webdriver.Chrome(r'C:\Users\89165\DidenkoResearch\personality-prediction-from-text\chromedriver_win32\chromedriver.exe')
# driver = webdriver.Chrome()
# driver= webdriver.Chrome(executable_path=r'C:\Users\89165\DidenkoResearch\personality-prediction-from-text\chromedriver_win32\chromedriver.exe')
# driver.get(interviews_href)
#This code will scroll down to the end
# while True:
#     try:
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         break
#     except: 
#         pass
# time.sleep(35)
# html = BeautifulSoup(driver.page_source)
# interviews_hrefs = find_articles(html, size)
interviews_hrefs = ['denbertech', 'starsync', 'publoft', 'benja', 'observa', 'weplate', 'design-in-dc', 'finmasters', 'tweet-hunter', 'woovly', 'basetemplates', 'sisu-academy', 'corebook', 'headlime', 'plutoview', 'fibery', 'wotnot', 'vimcal', 'bildr', 'reform', 'photobooth-supply', 'zenmaid', 'upvoty', 'gorails', 'tactiq', 'rize', 'tally', 'syria-airlift-project', 'microconf', 'zenoutreach', 'mubert', 'marketmuse', 'headway', 'inked-sports', 'crabi', 'snapshooter', 'photopea', 'siempo', 'akko', 'feedcheck', 'manyrequests', 'xena-intelligence', 'css-for-js-developers', 'zogics', 'lifewave', 'aplano', 'ombori', 'financer', 'hawkers', 'bizzbee', 'leilo', 'embarque', 'freshconnect', 'no-code-founders', 'endeavorun', 'frontend-mentor', 'resumemaker', 'locationrebel', 'flexiple-2021', 'edlatimore', 'mycity', 'blogging-for-devs', 'patron-ai', 'the-agent-nest', 'raw-gains', 'flexy', 'adproval', 'young-entrepreneurs-program', 'strengthrunning', 'viewst', 'growsurf', 'localtown', 'bounce', 'awesomic', 'swagup', 'mentorcruise', 'impossible', 'hello-tyro', 'refolo', 'copyai', 'kimp', 'pactero', 'refrens', 'qor360', 'sinoficina', 'openmargin', 'cam-ly', 'vacation-bird', 'scraper-api', 'cold-email-studio', 'ringdaddy', 'station', 'juice-startup', 'toki', 'vidico', 'reality-hunt', 'creator-growth-lab', 'salonist', 'lernin-games', 'career-sidekick', 'teacher-finder', 'the-careside', 'deluxemaid', 'formatically', 'nogood', 'newco', 'simplelogin', 'melon', 'my-auto-shop', 'circle', 'henry', 'stacking-the-bricks', 'visualsitemaps', 'eola', 'sofetch', 'linkody', 'uptrend', 'inovvorx', 'unicorn-platform', 'vulpine', 'softwareideas', 'plausible', 'kamua', 'zapstream', 'repairdesk', 'stronger-u', 'graphite-docs', 'habitual', 'pathways', 'buttercloth', 'thepresence', 'adleaf-technologies', 'colorado-mobile-drug-testing', 'watu', 'hawthorne-strategies', 'curiouscheck', 'rent-nest', 'scout', 'rent-round', 'swoop', 'kopely', 'helpwise', 'dotahaven', 'lunch-money', 'bunnyshell', 'tuff', 'cuddli', 'mailbrew', 'friday', 'processkit', '101-studios', 'ne-lounge', 'wavve', 'lemlist', 'teamometer', 'kaya-gs', 'dream-client-academy', 'habitify', 'motionthink', 'kolos', 'branch', 'scalefusion', 'price2spy', 'community-coders', 'yottio', 'patriot-chimney', 'ansaro', 'scrapingbee', 'swipes', 'crossnet', 'playdate', 'okami-pack', 'gawkbox', 'waydev', 'vo2-sportswear', 'sport-draftr', 'mealsurfers', 'tali', 'gulp', 'imaginovation', 'readership', 'flexiple', 'mangools', 'qwaiting', 'stone', 'mongoose-cricket', 'standuply', 'claimcompass', 'justreachout', '140-canvas', 'festivilia', 'oneup', 'singulution', 'encharge', 'narcine', 'nox', 'growthmentor', 'wedmap', 'campertunity', 'aura', 'profitabilly', 'rankd-seo', 'emit', 'tailor', 'hubrif', 'repitchbook', 'nerdpilots', 'growth-cave', 'genm', 'hyperstarter', 'pagestead', '10er', 'canary', 'combat-flip-flops', 'hubstaff', 'pull-reminders', 'lets-reach-success', 'seomator', 'metricspot', 'pear-cards', 'dick-at-your-door', 'one-hour-professor', '40-aprons', 'reroute-lifestyle', 'leadsbridge', 'beardbrand', 'posture-keeper', 'trackin', 'joker-greeting', 'browserless', 'urban-masterclass', 'iwantthatflight', 'park-io', 'matboard-and-more', 'scream-pretty', 'rayna-tours', 'onak', 'liveagent', 'puppet-pelts', 'gadget-flow', 'content-snare', 'bugfender', 'spuds', 'tandem-app', 'wantremotejob', 'lieferoo', 'twitch-highlights', 'jobridge', 'haptly', 'chowdy', 'zor-technology', 'the-punjab-kitchen', 'taleship', 'eventloot', 'hashtag-pirate', 'phez', 'muun', 'fantastic-house-buyers', 'wura', 'bediwin-information-services', 'ink', 'explorevr', 'autto-in', 'lockpick-entertainment', 'worldos', 'birdy', 'motobox', 'the-nerd-cave', 'knownet', 'legaats', 'onepagetrip', 'busymind', 'botnim', 'ropero', 'gameslog', 'abby', 'beehiveid', 'delite', 'phoenix', 'addressbin', 'vivalatina', 'mishra-motors', 'team-voice', 'quickhaggle', 'boston-apartment-hub', 'tandem', 'toygaroo', 'flux', 'hot-barber', 'brisk', 'eloquis', 'hitronasplet', 'sharkius', 'gymlisted', 'waterproof-digital-camera', 'asktina', 'the-blogging-manifesto', 'howell-market']
interviews_full_hrefs = [one_interview_href + interviews_hrefs[i] for i in range(0, len(interviews_hrefs))]
text_list = []
for href in interviews_full_hrefs:
    text_list.append([clear_text(href),href])
print("ready")

ready


In [18]:
main_url = 'https://www.failory.com'
interviews_href = main_url + '/interviews'
one_interview_href = main_url + '/interview/'
size = 203
interviews_hrefs = find_articles(interviews_href, size)
interviews_full_hrefs = [one_interview_href + interviews_hrefs[i] for i in range(0, len(interviews_hrefs))]
text_list = []
for href in interviews_full_hrefs:
    text_list.append([clear_text(href),href])

TypeError: slice indices must be integers or None or have an __index__ method

In [17]:
print(len(text_list))
predictions_list = []
for i in range(len(text_list)):
    predictions = predict_personality(text_list[i][0][0])
    predictions_list.append(predictions)
    predictions_list.append(text_list[i][0][1])
    print("predicted personality:", predictions, text_list[i][0][1], text_list[i][1])
# df = pd.DataFrame(dict(r=predictions, theta=['EXT','NEU','AGR', 'CON', 'OPN']))
# fig = px.line_polar(df, r='r', theta='theta', line_close=True)
# fig.show()

298
predicted personality: [0, 1, 0, 1, 1] 0 https://www.failory.com/interview/denbertech
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/starsync
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/publoft
predicted personality: [1, 1, 0, 1, 1] 0 https://www.failory.com/interview/benja
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/observa
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/weplate
predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/design-in-dc
predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/finmasters
predicted personality: [1, 1, 0, 1, 1] 1 https://www.failory.com/interview/tweet-hunter
predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/woovly
predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/basetemplates
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.co

predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/lernin-games
predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/career-sidekick
predicted personality: [1, 0, 0, 0, 1] 0 https://www.failory.com/interview/teacher-finder
predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/the-careside
predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/deluxemaid
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/formatically
predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/nogood
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/newco
predicted personality: [1, 1, 1, 1, 1] 1 https://www.failory.com/interview/simplelogin
predicted personality: [0, 1, 1, 1, 1] 0 https://www.failory.com/interview/melon
predicted personality: [1, 1, 1, 0, 1] 1 https://www.failory.com/interview/my-auto-shop
predicted personality: [1, 1, 0, 1, 1] 1 https://w

predicted personality: [1, 1, 0, 1, 1] 0 https://www.failory.com/interview/nox
predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/growthmentor
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/wedmap
predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/campertunity
predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/aura
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/profitabilly
predicted personality: [1, 1, 0, 1, 1] 1 https://www.failory.com/interview/rankd-seo
predicted personality: [1, 1, 0, 1, 1] 1 https://www.failory.com/interview/emit
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/tailor
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/hubrif
predicted personality: [1, 1, 0, 1, 1] 0 https://www.failory.com/interview/repitchbook
predicted personality: [0, 1, 1, 1, 1] 1 https://www.failory.com/interview/nerd

predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/asktina
predicted personality: [1, 1, 1, 0, 1] 0 https://www.failory.com/interview/the-blogging-manifesto
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/howell-market


In [81]:
for i in range(len(text_list)):
    predictions = predict_personality(text_list[i][0][0])
    print("predicted personality:", predictions, text_list[i][0][1], text_list[i][1])
# df = pd.DataFrame(dict(r=predictions, theta=['EXT','NEU','AGR', 'CON', 'OPN']))
# fig = px.line_polar(df, r='r', theta='theta', line_close=True)
# fig.show()

predicted personality: [0, 1, 0, 1, 1] 0 https://www.failory.com/interview/denbertech
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/starsync
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/publoft
predicted personality: [1, 1, 0, 1, 1] 0 https://www.failory.com/interview/benja
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/observa
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/interview/weplate
predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/design-in-dc
predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/finmasters
predicted personality: [1, 1, 0, 1, 1] 1 https://www.failory.com/interview/tweet-hunter
predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/woovly
predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/basetemplates
predicted personality: [1, 1, 0, 0, 1] 0 https://www.failory.com/in

predicted personality: [1, 1, 0, 0, 1] 1 https://www.failory.com/interview/career-sidekick


In [18]:
df=pd.DataFrame()

In [19]:
for i in range(len(text_list)):
    predictions_ext = predict_personality_ext(text_list[i][0][0])
    predictions_neu = predict_personality_neu(text_list[i][0][0])
    predictions_agr = predict_personality_agr(text_list[i][0][0])
    predictions_con = predict_personality_con(text_list[i][0][0])
    predictions_opn = predict_personality_opn(text_list[i][0][0])
    df.loc[i, 'ext'] = predictions_ext
    df.loc[i, 'neu'] = predictions_neu
    df.loc[i, 'agr'] = predictions_agr
    df.loc[i, 'con'] = predictions_con
    df.loc[i, 'opn'] = predictions_opn
    df.loc[i, 'success'] = text_list[i][0][1]
print(df)

     ext  neu  agr  con  opn  success
0    0.0  1.0  0.0  1.0  1.0      0.0
1    1.0  1.0  0.0  0.0  1.0      0.0
2    1.0  1.0  0.0  0.0  1.0      0.0
3    1.0  1.0  0.0  1.0  1.0      0.0
4    1.0  1.0  0.0  0.0  1.0      0.0
..   ...  ...  ...  ...  ...      ...
293  1.0  1.0  0.0  0.0  1.0      0.0
294  1.0  1.0  0.0  0.0  1.0      0.0
295  1.0  1.0  0.0  0.0  1.0      0.0
296  1.0  1.0  1.0  0.0  1.0      0.0
297  1.0  1.0  0.0  0.0  1.0      0.0

[298 rows x 6 columns]


In [20]:
df['success'] = df['success'].astype(int)
df['ext'] = df['ext'].astype(int)
df['neu'] = df['neu'].astype(int)
df['agr'] = df['agr'].astype(int)
df['con'] = df['con'].astype(int)
df['opn'] = df['opn'].astype(int)

In [21]:
print(df)

     ext  neu  agr  con  opn  success
0      0    1    0    1    1        0
1      1    1    0    0    1        0
2      1    1    0    0    1        0
3      1    1    0    1    1        0
4      1    1    0    0    1        0
..   ...  ...  ...  ...  ...      ...
293    1    1    0    0    1        0
294    1    1    0    0    1        0
295    1    1    0    0    1        0
296    1    1    1    0    1        0
297    1    1    0    0    1        0

[298 rows x 6 columns]


In [22]:
import numpy as np

In [23]:
ext_corr = pd.crosstab(df['ext'], df['success'])
ext_corr

success,0,1
ext,,
0,16,26
1,113,143


In [24]:
(ext_corr[0][0]*ext_corr[1][1]-ext_corr[1][0]*ext_corr[0][1])/np.sqrt((ext_corr[0][0]+ext_corr[1][0])*(ext_corr[0][0]+ext_corr[0][1])*(ext_corr[1][1]+ext_corr[1][0])*(ext_corr[1][1]+ext_corr[0][1]))

-0.04245516385100116

In [25]:
neu_corr = pd.crosstab(df['neu'], df['success'])
neu_corr

success,0,1
neu,,
0,2,0
1,127,169


In [26]:
(neu_corr[0][0]*neu_corr[1][1]-neu_corr[1][0]*neu_corr[0][1])/np.sqrt((neu_corr[0][0]+neu_corr[1][0])*(neu_corr[0][0]+neu_corr[0][1])*(neu_corr[1][1]+neu_corr[1][0])*(neu_corr[1][1]+neu_corr[0][1]))

0.09408440429181075

In [27]:
agr_corr = pd.crosstab(df['agr'], df['success'])
agr_corr

success,0,1
agr,,
0,111,148
1,18,21


In [28]:
(agr_corr[0][0]*agr_corr[1][1]-agr_corr[1][0]*agr_corr[0][1])/np.sqrt((agr_corr[0][0]+agr_corr[1][0])*(agr_corr[0][0]+agr_corr[0][1])*(agr_corr[1][1]+agr_corr[1][0])*(agr_corr[1][1]+agr_corr[0][1]))

-0.0224400510454732

In [29]:
con_corr = pd.crosstab(df['con'], df['success'])
con_corr

success,0,1
con,,
0,91,122
1,38,47


In [30]:
(con_corr[0][0]*con_corr[1][1]-con_corr[1][0]*con_corr[0][1])/np.sqrt((con_corr[0][0]+con_corr[1][0])*(con_corr[0][0]+con_corr[0][1])*(con_corr[1][1]+con_corr[1][0])*(con_corr[1][1]+con_corr[0][1]))

-0.018069951068747567

In [31]:
opn_corr = pd.crosstab(df['opn'], df['success'])
opn_corr

success,0,1
opn,,
1,129,169


In [38]:
(opn_corr[0][0]*opn_corr[1][1]-opn_corr[1][0]*opn_corr[0][1])/np.sqrt((opn_corr[0][0]+opn_corr[1][0])*(opn_corr[0][0]+opn_corr[0][1])*(opn_corr[1][1]+opn_corr[1][0])*(opn_corr[1][1]+opn_corr[0][1]))

KeyError: 0

In [41]:
df.to_csv('DidResearchenko.csv')

In [32]:
df1 = df.drop(columns=['success'], axis=1)

In [56]:
X=df1
y=df['success']

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [58]:
clf = LogisticRegression().fit(X_train, y_train)

In [60]:
y_pred = clf.predict(X_test)

In [61]:
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[ 0, 47],
       [ 2, 41]], dtype=int64)

In [53]:
clf.coef_

array([[-2.47943575e-01,  7.64105802e-01, -1.13215229e-01,
        -9.76388386e-02,  8.02403615e-05]])

In [63]:
from sklearn.metrics import classification_report
target_names = ['success', 'fail']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     success       0.00      0.00      0.00        47
        fail       0.47      0.95      0.63        43

    accuracy                           0.46        90
   macro avg       0.23      0.48      0.31        90
weighted avg       0.22      0.46      0.30        90



In [55]:
from sklearn.model_selection import train_test_split